# Exploratory analysis of inpatient hospital rates

To get the data, go to https://www.dolthub.com/repositories/dolthub/transparency-in-pricing and clone the repo, or do:

```
sudo curl -L https://github.com/dolthub/dolt/releases/latest/download/install.sh | sudo bash
dolt clone dolthub/transparency-in-pricing
```

to get a copy of dolt and immediately clone the database. It may take a while -- consider going for a walk, or calling your mom (she misses you.)

I do most of my data work with parquet files, so it's been easiest for me to just export the data to parquet, then read most of what I need from duckdb:

```
> dolt table export rate rate.parquet
```
Fire up duckdb and run a query just to get the rates with a non-null ms_drg.
```
./duckdb
duckdb> select hospital_id, description, local_code, ms_drg, standard_charge, standard_charge_percent, contracting_method, additional_generic_notes, additional_payer_specific_notes  FROM read_parquet('rate.parquet')  where ms_drg is not NULL ) TO 'msdrg.csv' (FORMAT CSV, HEADER);
```

In [11]:
import polars as pl
import pandas as pd # just needed for typing
df = pl.read_csv('msdrg.csv', infer_schema_length = 0)

In [12]:
df = df.select([
    'hospital_id',
    'description',
    'local_code',
    'ms_drg',
    'payer_name',
    'plan_name',
    'payer_category',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_generic_notes'
]).with_columns(pl.col('standard_charge').cast(float))

In [13]:
descs = df.groupby(['ms_drg']).agg(pl.first('description').str.to_uppercase())

# Checking the spread in price of different kinds of procedures

I wanted to know the DRGs that had the greatest spread, so I looked at the coefficient of variance, the standard deviation of the price, divided by the mean of the price, for each code.

In [233]:
spreads = (
    df
    # select the non-cash, non-gross, non-max/min charges
    .filter(pl.col('payer_category') == 'payer')
    
    # look only at the fee for service rates
    .filter(pl.col('contracting_method').is_null())
    .filter(pl.col('standard_charge').is_not_null())
    .filter(pl.col('standard_charge_percent').is_null())
    .filter(pl.col('standard_charge') > 0)
    
    # filter out outliers
    .with_columns([
        pl.col('standard_charge').quantile(0.1).over('ms_drg').alias('q1'),
        pl.col('standard_charge').quantile(0.9).over('ms_drg').alias('q3'),
    ])
    .filter(pl.col('standard_charge') > (pl.col('q1') - 1.5 * (pl.col('q3')-pl.col('q1'))))
    .filter(pl.col('standard_charge') < (pl.col('q3') + 1.5 * (pl.col('q3')-pl.col('q1'))))

    
    # compute the coefficient of variation
    .groupby(['ms_drg', 'q1', 'q3'])
    .agg([
        pl.col('standard_charge').mean().alias('mean_rate'),
        pl.col('standard_charge').std().alias('std_rate'),
        pl.col('hospital_id').n_unique().alias('available_at_n_hospitals'),
    ])
    .with_columns(
        (pl.col('std_rate') / pl.col('mean_rate')).alias('coefficient_of_variation')
    )
    
    # get descriptions
    .join(descs, on = 'ms_drg')
    
    # we want a decent sample size
    .filter(pl.col('available_at_n_hospitals') > 10)
    .sort('coefficient_of_variation')
    
).select(['ms_drg', 'description', 'q1', 'q3', 'coefficient_of_variation', 'available_at_n_hospitals'])

Let's look at the procedures with the most variation:

In [234]:
spreads.to_pandas().tail(5)

,ms_drg,description,q1,q3,coefficient_of_variation,available_at_n_hospitals
847,490,BACK & NECK PROC EXC SPINAL FUSION W CC/MCC OR...,1100.00,158640.20,1.086488,47
848,129,MAJOR HEAD & NECK PROCEDURES W CC/MCC OR MAJOR...,1180.00,309506.05,1.107105,49
849,134,"OTHER EAR, NOSE, MOUTH & THROAT O.R. PROCEDURE...",1050.00,103520.10,1.150363,60
850,986,PROSTATIC O.R. PROCEDURE UNRELATED TO PRINCIPA...,1173.72,120116.60,1.153488,45
851,007,LUNG TRANSPLANT,66921.68,643132.43,1.238857,159


And those with the least:

In [235]:
spreads.to_pandas().head(5)

,ms_drg,description,q1,q3,coefficient_of_variation,available_at_n_hospitals
0,591,ANOXIC & OTHER SEVERE BRAIN DAMAGE,5900.71,11202.56,0.303201,11
1,806,VAGINAL DELIVERY WITHOUT STERILIZATION OR D&C ...,4014.32,12592.99,0.478413,232
2,785,CESAREAN SECTION WITH STERILIZATION WITHOUT CC...,5289.98,16140.40,0.494063,228
3,138,MOUTH PROCEDURES WITHOUT CC/MCC,4872.70,14234.06,0.496893,204
4,117,INTRAOCULAR PROCEDURES WITHOUT CC/MCC,6004.33,17539.75,0.498615,194


Variation will be higher when the services are offered at few hospitals, so let's set a limit:

In [236]:
(
    spreads.filter(pl.col('available_at_n_hospitals') > 100)
    .to_pandas()
)

,ms_drg,description,q1,q3,coefficient_of_variation,available_at_n_hospitals
0,806,VAGINAL DELIVERY WITHOUT STERILIZATION OR D&C ...,4014.32,12592.99,0.478413,232
1,785,CESAREAN SECTION WITH STERILIZATION WITHOUT CC...,5289.98,16140.40,0.494063,228
2,138,MOUTH PROCEDURES WITHOUT CC/MCC,4872.70,14234.06,0.496893,204
3,117,INTRAOCULAR PROCEDURES WITHOUT CC/MCC,6004.33,17539.75,0.498615,194
4,804,OTHER O.R. PROCEDURES OF THE BLOOD AND BLOOD F...,7817.46,23031.79,0.498837,211
...,...,...,...,...,...,...
760,022,INTRACRANIAL VASCULAR PROCEDURES WITH PRINCIPA...,28086.07,138872.41,0.892821,196
761,006,LIVER TRANSPLANT WITHOUT MCC,27198.93,173785.41,0.952490,167
762,895,"ALCOHOL, DRUG ABUSE OR DEPENDENCE WITH REHABIL...",7491.33,59319.50,0.979773,159
763,969,HIV WITH EXTENSIVE O.R. PROCEDURES WITH MCC,33462.28,241020.78,1.043267,209


So it looks like lung transplants have the highest coefficient of variance, whereas normal vaginal deliveries have the lowest.

# Let's make some charts

In [237]:
## chart theming
%run urban.py

# plotting library
import altair as alt
alt.themes.register('urban', urban_theme)
alt.themes.enable('urban')

ThemeRegistry.enable('urban')

Importing the hospital data will allow us to color our charts by certain hospital properties (like rural/urban). To get the hospital data, I did

```
dolt table export hospital hospital_5_23.csv
```
while in the same directory as the `transparency-in-pricing` repo.

In [238]:
hosp = pl.read_csv('hospital_5_23.csv', infer_schema_length = 0)

You can see that we have lots of different data on the different hospitals, which will allow us to group by different data facets when we do our exploratory analysis.

In [239]:
hosp.head().to_pandas()

,id,ein,name,alt_name,system_name,addr,city,state,zip,phone,urban_rural,category,control_type,medicare_termination_status,last_updated,file_name,mrf_url,permalink,transparency_page,additional_notes
0,010011,63-0578923,ST VINCENT'S EAST,Ascension St. Vincent's East,None,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,2058383122,U,Short Term,FOR PROFIT - PARTNERSHIP,ACTIVE PROVIDER,2022-09-30,630578923_ascension-saint-vincents-east_standa...,https://healthcare.ascension.org/-/media/proje...,None,https://healthcare.ascension.org/price-transpa...,None
1,010056,63-0288864,ST VINCENT'S BIRMINGHAM,Ascension Saint Vincent's Birmingham,None,810 ST VINCENT'S DRIVE,BIRMINGHAM,AL,35205,2059397000,U,Short Term,FOR PROFIT - PARTNERSHIP,ACTIVE PROVIDER,2022-09-30,630288864_saint-vincents-birmingham_standardch...,https://healthcare.ascension.org/-/media/proje...,None,https://healthcare.ascension.org/price-transpa...,None
2,010130,63-1146531,ST VINCENT'S ST CLAIR,Ascension Saint Vincent's St. Clair,None,7063 VETERANS PARKWAY,PELL CITY,AL,35125,2053383301,U,Short Term,FOR PROFIT - PARTNERSHIP,ACTIVE PROVIDER,2022-09-30,631146531_saint-vincents-saint-clair_standardc...,https://healthcare.ascension.org/-/media/proje...,None,https://healthcare.ascension.org/price-transpa...,None
3,010173,81-0935368,ST VINCENT'S CHILTON,Ascension Saint Vincent's Chilton,None,2030 LAY DAM ROAD,CLANTON,AL,35045,2052584400,U,Short Term,FOR PROFIT - INDIVIDUAL,ACTIVE PROVIDER,2022-09-30,810935368_saint-vincents-chilton_standardcharg...,https://healthcare.ascension.org/-/media/proje...,None,https://healthcare.ascension.org/price-transpa...,None
4,011305,63-0909073,ST VINCENTS BLOUNT,Ascension St. Vincent's Blount,None,150 GILBREATH DRIVE,ONEONTA,AL,35121,2052743000,U,Critical Access Hospitals,FOR PROFIT - PARTNERSHIP,ACTIVE PROVIDER,2022-09-30,630909073_saint-vincents-blount_standardcharge...,https://healthcare.ascension.org/-/media/proje...,None,https://healthcare.ascension.org/price-transpa...,None


In [240]:
control_type = pl.col('control_type')
ur = pl.col('urban_rural')

hosp = hosp.with_columns([
    pl
    .when(control_type.str.contains('FOR PROFIT')).then('For-profit')
    .when(control_type.str.contains('NONPROFIT')).then('Nonprofit')
    .when(control_type.str.contains('GOVERNMENT')).then('Government')
    .when(control_type.is_null()).then('No data')
    .alias('type'),
    pl
    .when(ur == 'R').then('Rural').when(ur == 'U').then('Urban').otherwise('No data').alias('ur'),
])


Let's define a function to make some more charts.

In [241]:
df_cleaned = (
    df
    # true payer information
    .filter(pl.col('payer_category') == 'payer')
    # only fee-for-service (not per-diem)
    .filter(pl.col('contracting_method').is_null())
    # filter out garbage
    .filter(pl.col('standard_charge') > 0)
    .with_columns([
        pl.col('standard_charge').quantile(0.1).over('ms_drg').alias('q1'),
        pl.col('standard_charge').quantile(0.9).over('ms_drg').alias('q3'),
    ])
    .filter(pl.col('standard_charge') > (pl.col('q1') - 1.5 * (pl.col('q3')-pl.col('q1'))))
    .filter(pl.col('standard_charge') < (pl.col('q3') + 1.5 * (pl.col('q3')-pl.col('q1'))))
)

In [301]:
def sidebyside_normed(drg: str, df: pd.DataFrame, groupby: str, stepsize: int, title: str, cutoff: int = None, frac_labels = 2) -> alt.Chart:
    data = (
        df_cleaned
        .filter(pl.col('ms_drg') == drg)
        .groupby('hospital_id')
        .agg(pl.mean('standard_charge'))
        .join(hosp, left_on = 'hospital_id', right_on = 'id')
    ).to_pandas()
    
    title={'text': title,
            'subtitle': [
                descs.filter(pl.col('ms_drg') == drg)['description'][0],
                'Histogram normalized by group. Each histogram sums to one.',
                'This is how much the hospital gets from the insurance company, averaged over all companies',
                'Data from DoltHub.com: https://www.dolthub.com/repositories/dolthub/transparency-in-pricing'
            ]}
    
    sort_order = list(sorted(data[groupby].unique()))
    try:
        sort_order.remove('No data')
        sort_order.append('No data')
    except ValueError:
        pass
    
    if cutoff:
        bin_ = alt.Bin(extent=[0, cutoff], step=stepsize)
    else:
        bin_ = alt.Bin(step=stepsize)
    
    return (alt
            .Chart(data)
            .transform_bin('bins', 'standard_charge', bin=bin_)
            .transform_aggregate(count='count()', groupby=['bins', groupby])
            .transform_window(total='sum(count)', frame=[None, None], groupby=[groupby])
            .transform_calculate(normalized_count='datum.count / datum.total')
            .mark_bar()
            .encode(
                x=alt.X(
                    'bins:N', 
                    title='price for service', 
                    axis=alt.Axis(format='$,.0f', labelExpr=f'datum.value % {stepsize*frac_labels} ? null : datum.label'),
                ),
                y=alt.Y('normalized_count:Q', title='fraction of total'),
                xOffset=f'{groupby}:N',
                color= alt.Color(f'{groupby}:N', sort = sort_order),
            )
            .properties(
                title=title,
                width=600,
                height=400
            )
           )

sidebyside_normed(
    drg = '807', 
    df = df_cleaned,
    # cutoff = 100_000, 
    groupby = 'type', 
    stepsize = 1_000, 
    title = 'Price of a normal vaginal delivery'
)

alt.Chart(...)

In [302]:
def sidebyside(drg: str, df: pd.DataFrame, groupby: str, stepsize: int, title: str, cutoff: int = None, frac_labels = 2) -> alt.Chart:
    data = (
        df_cleaned
        .filter(pl.col('ms_drg') == drg)
        .groupby('hospital_id')
        .agg(pl.mean('standard_charge'))
        .join(hosp, left_on = 'hospital_id', right_on = 'id')
    ).to_pandas()
    
    title={'text': title,
            'subtitle': [
                descs.filter(pl.col('ms_drg') == drg)['description'][0],
                'This is how much the hospital gets from the insurance company, averaged over all companies',
                'Data from DoltHub.com: https://www.dolthub.com/repositories/dolthub/transparency-in-pricing'
            ]}
    
    sort_order = list(sorted(data[groupby].unique()))
    try:
        sort_order.remove('No data')
        sort_order.append('No data')
    except ValueError:
        pass
    
    if cutoff:
        bin_ = alt.Bin(extent=[0, cutoff], step=stepsize)
    else:
        bin_ = alt.Bin(step=stepsize)
        
    return (alt
            .Chart(data)
            .transform_bin('bins', 'standard_charge', bin=bin_)
            .mark_bar()
            .encode(
                x=alt.X(
                    'bins:N', 
                    title='price for service', 
                    axis=alt.Axis(format='$,.0f', labelExpr=f'datum.value % {stepsize*frac_labels} ? null : datum.label'),
                ),
                y=alt.Y('count():Q', title='counts'),
                xOffset=f'{groupby}:N',
                color= alt.Color(f'{groupby}:N', sort = sort_order),
            )
            .properties(
                title=title,
                width=600,
                height=400
            )
           )

sidebyside(
    drg = '807', 
    df = df_cleaned,
    cutoff = 100_000, 
    groupby = 'ur', 
    stepsize = 1_000, 
    title = 'Price of a normal vaginal delivery'
)

alt.Chart(...)

In [303]:
def stacked(drg: str, df: pd.DataFrame, groupby: str, stepsize: int, title: str, cutoff: int = None, frac_labels = 2) -> alt.Chart:
    data = (
        df_cleaned
        .filter(pl.col('ms_drg') == drg)
        .groupby('hospital_id')
        .agg(pl.mean('standard_charge'))
        .join(hosp, left_on = 'hospital_id', right_on = 'id')
    ).to_pandas()
    
    title={'text': title,
            'subtitle': [
                descs.filter(pl.col('ms_drg') == drg)['description'][0],
                'This is how much the hospital gets from the insurance company, averaged over all companies',
                'Data from DoltHub.com: https://www.dolthub.com/repositories/dolthub/transparency-in-pricing'
            ]}
    
    sort_order = list(sorted(data[groupby].unique()))
    try:
        sort_order.remove('No data')
        sort_order.append('No data')
    except ValueError:
        pass
    
    if cutoff:
        bin_ = alt.Bin(extent=[0, cutoff], step=stepsize)
    else:
        bin_ = alt.Bin(step=stepsize)
    
    return alt.Chart(data).mark_bar().encode(
        alt.X(
            'standard_charge:Q', 
            bin=bin_,
            title='price for service',
            axis=alt.Axis(format='$,.0f', labelExpr=f'datum.value % {stepsize*frac_labels} ? null : datum.label'),
        ),
        alt.Y('count():Q', title='counts'),
        color= alt.Color(groupby, sort = sort_order),
    ).properties(
        title = title,
        width=600,
        height=400
    )

stacked(
    drg = '807', 
    df = df_cleaned,
    # cutoff = 13_000, 
    groupby = 'type', 
    stepsize = 500, 
    title = 'Price of a normal vaginal delivery',
    frac_labels = 4
)

alt.Chart(...)

What kinds of hospitals are more expensive on average?

In [431]:
def compare_profitnonprofit(slice_start, slice_end, title, drg_filter):
    source = (
        df_cleaned.drop('description')
        .join(descs, on = 'ms_drg')
        .filter(pl.col('description').str.contains(drg_filter))
        .join(hosp, left_on = 'hospital_id', right_on = 'id')
        .groupby(['ms_drg', 'type']).agg([
            pl.col('standard_charge').mean()
        ])
        .pivot(index = 'ms_drg', columns = 'type', values = 'standard_charge', aggregate_function = 'first')
        .with_columns(
            (pl.col('Nonprofit')/pl.col('For-profit')).alias('ratio')
        )
        .select(['ms_drg', 'ratio'])
        .filter(pl.col('ms_drg').is_not_null())
        .filter(pl.col('ratio').is_not_null())
        .sort('ms_drg')[slice_start:slice_end]
        .join(descs, on = 'ms_drg')
        .to_pandas()
    )
    
    title={'text': title,
        'subtitle': [
            f'For all DRG codes that contain the word: {drg_filter}',
            'This is how much the hospital gets from the insurance company, per procedure',
            'Data from DoltHub.com: https://www.dolthub.com/repositories/dolthub/transparency-in-pricing'
        ]}


    return alt.Chart(source).mark_bar().encode(
        x=alt.X(
            "description:N", 
            title = 'inpatient DRG code (procedure)', 
            sort='ascending', 
            axis=alt.Axis(labelAngle=-45, labelLimit = 200,)
        ),
        y=alt.Y("ratio:Q", title = 'ratio of non-profit to for-profit rate'),
        color=alt.condition(
            alt.datum.ratio > 1,
            alt.value("#1696d2"),  # The positive color
            alt.value("black")  # The negative color
        )
    ).properties(width=800, title = title)

In [429]:
compare_profitnonprofit(0,25, title = 'Nonprofits can be costlier, in some cases, than for-profits', drg_filter = '')

alt.Chart(...)

In [432]:
compare_profitnonprofit(0,25, title = 'Nonprofits can be costlier, in some cases, than for-profits', drg_filter = 'TRANSPLANT')

alt.Chart(...)

In [485]:
compare_profitnonprofit(0,25, title = 'For-profits exceed costs of nonprofits for cardiac procedures', drg_filter = 'CARDIAC')

alt.Chart(...)

In [484]:
compare_profitnonprofit(0,25, title = 'Nonprofits can be costlier, in some cases, than for-profits', drg_filter = 'LEUKEMIA')

alt.Chart(...)

In [442]:
from collections import Counter

In [481]:
Counter(' '.join([x for x in ''.join(list([x for x in descs['description'] if x])).split(' ') if len(x) > 5]).split(' ')).most_common(20)

[('WITHOUT', 322),
 ('PROCEDURES', 268),
 ('SYSTEM', 89),
 ('DISORDERS', 89),
 ('EXCEPT', 72),
 ('DIAGNOSES', 46),
 ('MCCOTHER', 39),
 ('DIAGNOSIS', 39),
 ('TISSUE', 38),
 ('PRINCIPAL', 31),
 ('CC/MCCOTHER', 28),
 ('MALIGNANCY', 25),
 ('CARDIAC', 25),
 ('MUSCULOSKELETAL', 24),
 ('REPRODUCTIVE', 24),
 ('CONNECTIVE', 23),
 ('CCOTHER', 23),
 ('LEUKEMIA', 21),
 ('INFECTIONS', 20),
 ('CATHETERIZATION', 18)]